In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# ============================================
# VERIFICADOR E UNIFICADOR DE ARQUIVOS CSV
# ============================================

class CSVMerger:
    """Classe para verificar e unir arquivos CSV"""
    
    def __init__(self, arquivo1, arquivo2, separador='|'):
        """
        Inicializa o verificador/unificador
        
        Args:
            arquivo1: Caminho do primeiro arquivo CSV
            arquivo2: Caminho do segundo arquivo CSV
            separador: Separador usado nos CSVs (padrão: |)
        """
        self.arquivo1 = arquivo1
        self.arquivo2 = arquivo2
        self.separador = separador
        self.df1 = None
        self.df2 = None
        self.df_merged = None
        self.compativel = False
        
    def carregar_arquivos(self):
        """Carrega os dois arquivos CSV"""
        print("📂 CARREGANDO ARQUIVOS...")
        print("=" * 60)
        
        try:
            # Tentar diferentes encodings se necessário
            encodings = ['utf-8', 'latin-1', 'iso-8859-1', 'cp1252']
            
            for encoding in encodings:
                try:
                    self.df1 = pd.read_csv(self.arquivo1, sep=self.separador, encoding=encoding)
                    print(f"✅ Arquivo 1 carregado com encoding {encoding}")
                    print(f"   📁 {self.arquivo1}")
                    print(f"   📊 Dimensões: {len(self.df1)} linhas x {len(self.df1.columns)} colunas")
                    break
                except:
                    continue
                    
            for encoding in encodings:
                try:
                    self.df2 = pd.read_csv(self.arquivo2, sep=self.separador, encoding=encoding)
                    print(f"✅ Arquivo 2 carregado com encoding {encoding}")
                    print(f"   📁 {self.arquivo2}")
                    print(f"   📊 Dimensões: {len(self.df2)} linhas x {len(self.df2.columns)} colunas")
                    break
                except:
                    continue
                    
            if self.df1 is None or self.df2 is None:
                raise Exception("Não foi possível carregar os arquivos")
                
            return True
            
        except Exception as e:
            print(f"❌ Erro ao carregar arquivos: {e}")
            return False
    
    def analisar_estrutura(self):
        """Analisa e compara a estrutura dos dois CSVs"""
        print("\n📋 ANÁLISE DA ESTRUTURA")
        print("=" * 60)
        
        # Colunas
        cols1 = set(self.df1.columns)
        cols2 = set(self.df2.columns)
        
        print("\n🔤 COLUNAS:")
        print(f"Arquivo 1: {len(cols1)} colunas")
        print(f"Arquivo 2: {len(cols2)} colunas")
        
        # Colunas em comum
        colunas_comuns = cols1.intersection(cols2)
        print(f"\n✅ Colunas em comum: {len(colunas_comuns)}")
        if len(colunas_comuns) <= 20:
            for col in sorted(colunas_comuns):
                print(f"   • {col}")
        else:
            for i, col in enumerate(sorted(colunas_comuns)[:10]):
                print(f"   • {col}")
            print(f"   ... e mais {len(colunas_comuns) - 10} colunas")
        
        # Colunas exclusivas
        exclusivas1 = cols1 - cols2
        exclusivas2 = cols2 - cols1
        
        if exclusivas1:
            print(f"\n⚠️ Colunas exclusivas do Arquivo 1: {len(exclusivas1)}")
            for col in sorted(exclusivas1)[:10]:
                print(f"   • {col}")
            if len(exclusivas1) > 10:
                print(f"   ... e mais {len(exclusivas1) - 10} colunas")
                
        if exclusivas2:
            print(f"\n⚠️ Colunas exclusivas do Arquivo 2: {len(exclusivas2)}")
            for col in sorted(exclusivas2)[:10]:
                print(f"   • {col}")
            if len(exclusivas2) > 10:
                print(f"   ... e mais {len(exclusivas2) - 10} colunas")
        
        # Verificar compatibilidade
        print("\n🔍 VERIFICAÇÃO DE COMPATIBILIDADE:")
        print("-" * 40)
        
        if cols1 == cols2:
            print("✅ Estruturas IDÊNTICAS - União simples possível")
            self.compativel = True
            self.tipo_uniao = "concat"
        elif len(colunas_comuns) > 0:
            print(f"⚠️ Estruturas PARCIALMENTE compatíveis")
            print(f"   {len(colunas_comuns)} colunas em comum podem ser unidas")
            self.compativel = True
            self.tipo_uniao = "merge"
        else:
            print("❌ Estruturas INCOMPATÍVEIS - Nenhuma coluna em comum")
            self.compativel = False
            self.tipo_uniao = None
            
        return self.compativel
    
    def verificar_tipos_dados(self):
        """Verifica compatibilidade dos tipos de dados nas colunas comuns"""
        print("\n📊 TIPOS DE DADOS NAS COLUNAS COMUNS:")
        print("=" * 60)
        
        colunas_comuns = set(self.df1.columns).intersection(set(self.df2.columns))
        
        incompativeis = []
        for col in sorted(colunas_comuns)[:20]:  # Verificar primeiras 20 colunas
            tipo1 = self.df1[col].dtype
            tipo2 = self.df2[col].dtype
            
            if tipo1 != tipo2:
                incompativeis.append({
                    'coluna': col,
                    'tipo_arquivo1': str(tipo1),
                    'tipo_arquivo2': str(tipo2)
                })
                print(f"⚠️ {col}:")
                print(f"   Arquivo 1: {tipo1}")
                print(f"   Arquivo 2: {tipo2}")
        
        if not incompativeis:
            print("✅ Todos os tipos de dados são compatíveis!")
        else:
            print(f"\n⚠️ {len(incompativeis)} colunas com tipos diferentes")
            print("   (Serão convertidas automaticamente na união)")
            
        return incompativeis
    
    def estatisticas_detalhadas(self):
        """Mostra estatísticas detalhadas dos dois arquivos"""
        print("\n📈 ESTATÍSTICAS DETALHADAS:")
        print("=" * 60)
        
        for i, df in enumerate([self.df1, self.df2], 1):
            print(f"\n📁 ARQUIVO {i}:")
            print("-" * 40)
            print(f"Total de linhas: {len(df):,}")
            print(f"Total de colunas: {len(df.columns)}")
            print(f"Total de células: {df.size:,}")
            print(f"Memória utilizada: {df.memory_usage(deep=True).sum() / 1024 / 1024:.2f} MB")
            
            # Valores nulos
            nulos = df.isnull().sum().sum()
            if nulos > 0:
                print(f"Valores nulos: {nulos:,} ({nulos/df.size*100:.2f}%)")
            
            # Valores duplicados
            duplicados = df.duplicated().sum()
            if duplicados > 0:
                print(f"Linhas duplicadas: {duplicados:,} ({duplicados/len(df)*100:.2f}%)")
            
            # Amostra de dados
            print(f"\n🔍 Primeiras 3 linhas:")
            print(df.head(3).to_string(max_cols=5))
    
    def unir_arquivos(self, metodo='auto', remover_duplicados=True):
        """
        Une os dois arquivos CSV
        
        Args:
            metodo: 'auto', 'concat', 'merge', 'outer'
            remover_duplicados: Remove linhas duplicadas após união
        
        Returns:
            DataFrame unificado
        """
        if not self.compativel:
            print("❌ Arquivos incompatíveis para união!")
            return None
            
        print("\n🔗 UNINDO ARQUIVOS...")
        print("=" * 60)
        
        try:
            if metodo == 'auto':
                metodo = self.tipo_uniao
            
            if metodo == 'concat' or set(self.df1.columns) == set(self.df2.columns):
                # União simples (estruturas idênticas)
                print("📌 Método: Concatenação simples (append)")
                # Adicionar coluna de origem
                self.df1['origem_arquivo'] = 'arquivo_1'
                self.df2['origem_arquivo'] = 'arquivo_2'
                self.df_merged = pd.concat([self.df1, self.df2], ignore_index=True)
                
            else:
                # União com merge (estruturas diferentes)
                print("📌 Método: União externa (outer join)")
                # Adicionar coluna de origem antes do merge
                self.df1['origem_arquivo'] = 'arquivo_1'
                self.df2['origem_arquivo'] = 'arquivo_2'
                self.df_merged = pd.concat([self.df1, self.df2], ignore_index=True, sort=False)
            
            print(f"✅ União concluída!")
            print(f"   Total de linhas: {len(self.df_merged):,}")
            print(f"   Total de colunas: {len(self.df_merged.columns)}")
            
            # Remover duplicados se solicitado
            if remover_duplicados:
                antes = len(self.df_merged)
                # Não considerar a coluna origem_arquivo para detectar duplicados
                cols_para_verificar = [col for col in self.df_merged.columns if col != 'origem_arquivo']
                self.df_merged = self.df_merged.drop_duplicates(subset=cols_para_verificar, keep='first')
                depois = len(self.df_merged)
                if antes > depois:
                    print(f"   🗑️ {antes - depois:,} linhas duplicadas removidas")
            
            # Estatísticas finais
            print(f"\n📊 RESULTADO FINAL:")
            print(f"   Linhas do arquivo 1: {len(self.df1):,}")
            print(f"   Linhas do arquivo 2: {len(self.df2):,}")
            print(f"   Linhas no arquivo unificado: {len(self.df_merged):,}")
            print(f"   Colunas no arquivo unificado: {len(self.df_merged.columns)}")
            
            return self.df_merged
            
        except Exception as e:
            print(f"❌ Erro ao unir arquivos: {e}")
            return None
    
    def salvar_resultado(self, nome_arquivo="ativos e inativos_full.csv", separador='|'):
        """
        Salva o arquivo unificado
        
        Args:
            nome_arquivo: Nome do arquivo de saída
            separador: Separador a usar no arquivo de saída
        """
        if self.df_merged is None:
            print("❌ Nenhum arquivo unificado para salvar!")
            return False
            
        try:
            print(f"\n💾 SALVANDO ARQUIVO UNIFICADO...")
            print("=" * 60)
            
            self.df_merged.to_csv(nome_arquivo, sep=separador, index=False, encoding='utf-8')
            
            # Verificar arquivo salvo
            tamanho = Path(nome_arquivo).stat().st_size / 1024 / 1024
            
            print(f"✅ Arquivo salvo com sucesso!")
            print(f"   📁 Nome: {nome_arquivo}")
            print(f"   📊 Tamanho: {tamanho:.2f} MB")
            print(f"   📝 Linhas: {len(self.df_merged):,}")
            print(f"   📋 Colunas: {len(self.df_merged.columns)}")
            print(f"   🔤 Separador: '{separador}'")
            
            return True
            
        except Exception as e:
            print(f"❌ Erro ao salvar arquivo: {e}")
            return False
    
    def executar_processo_completo(self):
        """Executa o processo completo de verificação e união"""
        print("🚀 INICIANDO PROCESSO DE VERIFICAÇÃO E UNIÃO DE CSVs")
        print("=" * 60)
        
        # 1. Carregar arquivos
        if not self.carregar_arquivos():
            return False
        
        # 2. Analisar estrutura
        if not self.analisar_estrutura():
            print("\n❌ Processo interrompido: arquivos incompatíveis")
            return False
        
        # 3. Verificar tipos de dados
        self.verificar_tipos_dados()
        
        # 4. Mostrar estatísticas
        self.estatisticas_detalhadas()
        
        # 5. Unir arquivos
        if self.unir_arquivos():
            # 6. Salvar resultado
            self.salvar_resultado()
            print("\n✨ PROCESSO CONCLUÍDO COM SUCESSO!")
            return True
        
        return False

# ============================================
# FUNÇÕES AUXILIARES
# ============================================

def verificar_e_unir_csvs(arquivo1, arquivo2, separador='|', nome_saida="ativos e inativos_full.csv"):
    """
    Função simplificada para verificar e unir dois CSVs
    
    Args:
        arquivo1: Caminho do primeiro arquivo
        arquivo2: Caminho do segundo arquivo
        separador: Separador usado nos CSVs
        nome_saida: Nome do arquivo de saída
    
    Returns:
        DataFrame unificado ou None se incompatível
    """
    merger = CSVMerger(arquivo1, arquivo2, separador)
    if merger.executar_processo_completo():
        return merger.df_merged
    return None

def analisar_multiplos_csvs(*arquivos, separador='|'):
    """
    Analisa múltiplos arquivos CSV para verificar compatibilidade
    
    Args:
        *arquivos: Lista de caminhos de arquivos
        separador: Separador usado nos CSVs
    """
    print("📊 ANÁLISE DE MÚLTIPLOS ARQUIVOS CSV")
    print("=" * 60)
    
    dataframes = []
    estruturas = []
    
    for i, arquivo in enumerate(arquivos, 1):
        try:
            df = pd.read_csv(arquivo, sep=separador)
            dataframes.append(df)
            estruturas.append(set(df.columns))
            print(f"✅ Arquivo {i}: {arquivo}")
            print(f"   Linhas: {len(df):,}, Colunas: {len(df.columns)}")
        except Exception as e:
            print(f"❌ Erro no arquivo {i}: {e}")
            return None
    
    # Verificar compatibilidade geral
    colunas_comuns = set.intersection(*estruturas)
    print(f"\n📋 Colunas em comum em TODOS os arquivos: {len(colunas_comuns)}")
    
    if len(colunas_comuns) > 0:
        print("✅ Arquivos podem ser unidos!")
        return dataframes
    else:
        print("❌ Arquivos incompatíveis para união")
        return None



In [2]:
# ============================================
# EXEMPLO DE USO
# ============================================

# Exemplo básico - SUBSTITUA COM SEUS ARQUIVOS
print("📝 EXEMPLO DE USO:")
print("=" * 60)
print("""
# Para usar com seus arquivos, execute:

merger = CSVMerger(r"D:\____________________FAPERGS\DADOS\1_dados_limpos\completo_ativos_com_sobreviveu.csv", r"D:\____________________FAPERGS\DADOS\1_dados_limpos\completo_inativos_nao_sobreviveu_pipe.csv", separador='|')
merger.executar_processo_completo()

# Ou use a função simplificada:
df_unificado = verificar_e_unir_csvs(
    'caminho/para/arquivo1.csv',
    'caminho/para/arquivo2.csv',
    separador='|',
    nome_saida='ativos e inativos_full.csv'
)
""")

merger = CSVMerger(r"D:\____________________FAPERGS\DADOS\1_dados_limpos\completo_ativos_com_sobreviveu.csv", r"D:\____________________FAPERGS\DADOS\1_dados_limpos\completo_inativos_nao_sobreviveu_pipe.csv", separador='|')
merger.executar_processo_completo()


# ============================================
# CRIAR ARQUIVOS DE EXEMPLO PARA TESTE
# ============================================

def criar_arquivos_exemplo():
    """Cria arquivos de exemplo para teste"""
    
    # Criar arquivo 1 - Ativos
    df_ativos = pd.DataFrame({
        'cnpj': ['06.303.914/0001-08', '11.222.333/0001-44', '99.888.777/0001-66'],
        'username': ['gruporezor', 'empresa2', 'empresa3'],
        'name': ['Grupo Rezor', 'Empresa 2', 'Empresa 3'],
        'followers_count': [955, 1200, 450],
        'status': ['ativo', 'ativo', 'ativo'],
        'timestamp': ['2025-06-04T18:04:48+0000', '2025-06-05T10:00:00+0000', '2025-06-06T14:30:00+0000']
    })
    
    # Criar arquivo 2 - Inativos
    df_inativos = pd.DataFrame({
        'cnpj': ['55.444.333/0001-22', '77.666.555/0001-11'],
        'username': ['empresa4', 'empresa5'],
        'name': ['Empresa 4', 'Empresa 5'],
        'followers_count': [200, 150],
        'status': ['inativo', 'inativo'],
        'timestamp': ['2025-01-15T09:00:00+0000', '2025-02-20T11:30:00+0000']
    })
    
    # Salvar arquivos de exemplo
    df_ativos.to_csv('exemplo_ativos.csv', sep='|', index=False)
    df_inativos.to_csv('exemplo_inativos.csv', sep='|', index=False)
    
    print("📁 Arquivos de exemplo criados:")
    print("   • exemplo_ativos.csv")
    print("   • exemplo_inativos.csv")
    print("\n🚀 Execute o código abaixo para testá-los:")
    print("merger = CSVMerger('exemplo_ativos.csv', 'exemplo_inativos.csv', '|')")
    print("merger.executar_processo_completo()")
    
    return df_ativos, df_inativos

# Descomentar para criar arquivos de exemplo
# df_ativos, df_inativos = criar_arquivos_exemplo()

print("\n✨ Código pronto para uso! Substitua os nomes dos arquivos e execute.")

📝 EXEMPLO DE USO:

# Para usar com seus arquivos, execute:

merger = CSVMerger(r"D:\____________________FAPERGS\DADOS_dados_limpos\completo_ativos_com_sobreviveu.csv", r"D:\____________________FAPERGS\DADOS_dados_limpos\completo_inativos_nao_sobreviveu_pipe.csv", separador='|')
merger.executar_processo_completo()

# Ou use a função simplificada:
df_unificado = verificar_e_unir_csvs(
    'caminho/para/arquivo1.csv',
    'caminho/para/arquivo2.csv',
    separador='|',
    nome_saida='ativos e inativos_full.csv'
)

🚀 INICIANDO PROCESSO DE VERIFICAÇÃO E UNIÃO DE CSVs
📂 CARREGANDO ARQUIVOS...
✅ Arquivo 1 carregado com encoding utf-8
   📁 D:\____________________FAPERGS\DADOS\1_dados_limpos\completo_ativos_com_sobreviveu.csv
   📊 Dimensões: 1292775 linhas x 15 colunas
✅ Arquivo 2 carregado com encoding utf-8
   📁 D:\____________________FAPERGS\DADOS\1_dados_limpos\completo_inativos_nao_sobreviveu_pipe.csv
   📊 Dimensões: 239713 linhas x 2 colunas

📋 ANÁLISE DA ESTRUTURA

🔤 COLUNAS:
Arquivo 

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [3]:
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# ============================================
# VERIFICADOR E UNIFICADOR DE ARQUIVOS CSV
# ============================================

class CSVMerger:
    """Classe para verificar e unir arquivos CSV"""
    
    def __init__(self, arquivo1, arquivo2, separador='|'):
        """
        Inicializa o verificador/unificador
        
        Args:
            arquivo1: Caminho do primeiro arquivo CSV
            arquivo2: Caminho do segundo arquivo CSV
            separador: Separador usado nos CSVs (padrão: |)
        """
        self.arquivo1 = arquivo1
        self.arquivo2 = arquivo2
        self.separador = separador
        self.df1 = None
        self.df2 = None
        self.df_merged = None
        self.compativel = False
        
    def carregar_arquivos(self):
        """Carrega os dois arquivos CSV"""
        print("📂 CARREGANDO ARQUIVOS...")
        print("=" * 60)
        
        try:
            # Tentar diferentes encodings se necessário
            encodings = ['utf-8', 'latin-1', 'iso-8859-1', 'cp1252']
            
            # Carregar primeiro arquivo
            for encoding in encodings:
                try:
                    self.df1 = pd.read_csv(self.arquivo1, sep=self.separador, encoding=encoding, low_memory=False)
                    print(f"✅ Arquivo 1 carregado com encoding {encoding}")
                    print(f"   📁 {Path(self.arquivo1).name}")
                    print(f"   📊 Dimensões: {len(self.df1):,} linhas x {len(self.df1.columns)} colunas")
                    break
                except UnicodeDecodeError:
                    continue
                except Exception as e:
                    if encoding == encodings[-1]:  # Se é o último encoding
                        print(f"   ❌ Erro: {e}")
                    continue
            
            # Carregar segundo arquivo        
            for encoding in encodings:
                try:
                    self.df2 = pd.read_csv(self.arquivo2, sep=self.separador, encoding=encoding, low_memory=False)
                    print(f"✅ Arquivo 2 carregado com encoding {encoding}")
                    print(f"   📁 {Path(self.arquivo2).name}")
                    print(f"   📊 Dimensões: {len(self.df2):,} linhas x {len(self.df2.columns)} colunas")
                    break
                except UnicodeDecodeError:
                    continue
                except Exception as e:
                    if encoding == encodings[-1]:  # Se é o último encoding
                        print(f"   ❌ Erro: {e}")
                    continue
                    
            if self.df1 is None or self.df2 is None:
                raise Exception("Não foi possível carregar um ou ambos os arquivos")
                
            return True
            
        except Exception as e:
            print(f"❌ Erro ao carregar arquivos: {e}")
            return False
    
    def analisar_estrutura(self):
        """Analisa e compara a estrutura dos dois CSVs"""
        print("\n📋 ANÁLISE DA ESTRUTURA")
        print("=" * 60)
        
        # Limpar nomes das colunas (remover espaços extras)
        self.df1.columns = self.df1.columns.str.strip()
        self.df2.columns = self.df2.columns.str.strip()
        
        # Colunas
        cols1 = set(self.df1.columns)
        cols2 = set(self.df2.columns)
        
        print(f"\n🔤 COLUNAS:")
        print(f"Arquivo 1: {len(cols1)} colunas")
        print(f"Arquivo 2: {len(cols2)} colunas")
        
        # Colunas em comum
        colunas_comuns = cols1.intersection(cols2)
        print(f"\n✅ Colunas em comum: {len(colunas_comuns)}")
        if len(colunas_comuns) <= 20:
            for col in sorted(colunas_comuns):
                print(f"   • {col}")
        else:
            for i, col in enumerate(sorted(colunas_comuns)[:10]):
                print(f"   • {col}")
            print(f"   ... e mais {len(colunas_comuns) - 10} colunas")
        
        # Colunas exclusivas
        exclusivas1 = cols1 - cols2
        exclusivas2 = cols2 - cols1
        
        if exclusivas1:
            print(f"\n⚠️ Colunas exclusivas do Arquivo 1: {len(exclusivas1)}")
            for col in list(sorted(exclusivas1))[:10]:
                print(f"   • {col}")
            if len(exclusivas1) > 10:
                print(f"   ... e mais {len(exclusivas1) - 10} colunas")
                
        if exclusivas2:
            print(f"\n⚠️ Colunas exclusivas do Arquivo 2: {len(exclusivas2)}")
            for col in list(sorted(exclusivas2))[:10]:
                print(f"   • {col}")
            if len(exclusivas2) > 10:
                print(f"   ... e mais {len(exclusivas2) - 10} colunas")
        
        # Verificar compatibilidade
        print("\n🔍 VERIFICAÇÃO DE COMPATIBILIDADE:")
        print("-" * 40)
        
        if cols1 == cols2:
            print("✅ Estruturas IDÊNTICAS - União simples possível")
            self.compativel = True
            self.tipo_uniao = "concat"
        elif len(colunas_comuns) > 0:
            print(f"⚠️ Estruturas PARCIALMENTE compatíveis")
            print(f"   {len(colunas_comuns)} colunas em comum podem ser unidas")
            self.compativel = True
            self.tipo_uniao = "merge"
        else:
            print("❌ Estruturas INCOMPATÍVEIS - Nenhuma coluna em comum")
            self.compativel = False
            self.tipo_uniao = None
            
        return self.compativel
    
    def verificar_tipos_dados(self):
        """Verifica compatibilidade dos tipos de dados nas colunas comuns"""
        print("\n📊 TIPOS DE DADOS NAS COLUNAS COMUNS:")
        print("=" * 60)
        
        colunas_comuns = set(self.df1.columns).intersection(set(self.df2.columns))
        
        if not colunas_comuns:
            print("⚠️ Nenhuma coluna em comum para verificar tipos")
            return []
        
        incompativeis = []
        colunas_verificar = list(sorted(colunas_comuns))[:20]  # Verificar primeiras 20 colunas
        
        for col in colunas_verificar:
            tipo1 = self.df1[col].dtype
            tipo2 = self.df2[col].dtype
            
            if tipo1 != tipo2:
                incompativeis.append({
                    'coluna': col,
                    'tipo_arquivo1': str(tipo1),
                    'tipo_arquivo2': str(tipo2)
                })
                print(f"⚠️ {col}:")
                print(f"   Arquivo 1: {tipo1}")
                print(f"   Arquivo 2: {tipo2}")
        
        if not incompativeis:
            print("✅ Todos os tipos de dados são compatíveis!")
        else:
            print(f"\n⚠️ {len(incompativeis)} colunas com tipos diferentes")
            print("   (Serão convertidas automaticamente na união)")
            
        return incompativeis
    
    def estatisticas_detalhadas(self):
        """Mostra estatísticas detalhadas dos dois arquivos"""
        print("\n📈 ESTATÍSTICAS DETALHADAS:")
        print("=" * 60)
        
        for i, df in enumerate([self.df1, self.df2], 1):
            print(f"\n📁 ARQUIVO {i}:")
            print("-" * 40)
            print(f"Total de linhas: {len(df):,}")
            print(f"Total de colunas: {len(df.columns)}")
            print(f"Total de células: {df.size:,}")
            print(f"Memória utilizada: {df.memory_usage(deep=True).sum() / 1024 / 1024:.2f} MB")
            
            # Valores nulos
            nulos = df.isnull().sum().sum()
            if nulos > 0:
                print(f"Valores nulos: {nulos:,} ({nulos/df.size*100:.2f}%)")
            
            # Valores duplicados
            duplicados = df.duplicated().sum()
            if duplicados > 0:
                print(f"Linhas duplicadas: {duplicados:,} ({duplicados/len(df)*100:.2f}%)")
            
            # Amostra de dados
            print(f"\n🔍 Primeiras 3 linhas (primeiras 5 colunas):")
            colunas_amostra = list(df.columns)[:5]
            print(df[colunas_amostra].head(3).to_string())
    
    def unir_arquivos(self, metodo='auto', remover_duplicados=True):
        """
        Une os dois arquivos CSV
        
        Args:
            metodo: 'auto', 'concat', 'merge', 'outer'
            remover_duplicados: Remove linhas duplicadas após união
        
        Returns:
            Boolean indicando sucesso da operação
        """
        if not self.compativel:
            print("❌ Arquivos incompatíveis para união!")
            return False
            
        print("\n🔗 UNINDO ARQUIVOS...")
        print("=" * 60)
        
        try:
            if metodo == 'auto':
                metodo = self.tipo_uniao
            
            # Fazer cópias para não modificar os originais
            df1_copy = self.df1.copy()
            df2_copy = self.df2.copy()
            
            # Adicionar coluna de origem
            df1_copy['origem_arquivo'] = Path(self.arquivo1).name
            df2_copy['origem_arquivo'] = Path(self.arquivo2).name
            
            if metodo == 'concat' or set(self.df1.columns) == set(self.df2.columns):
                # União simples (estruturas idênticas ou muito similares)
                print("📌 Método: Concatenação simples (append)")
                self.df_merged = pd.concat([df1_copy, df2_copy], ignore_index=True, sort=False)
                
            else:
                # União com outer join (estruturas diferentes)
                print("📌 Método: União externa (outer join)")
                self.df_merged = pd.concat([df1_copy, df2_copy], ignore_index=True, sort=False)
            
            print(f"✅ União concluída!")
            print(f"   Total de linhas inicial: {len(self.df_merged):,}")
            print(f"   Total de colunas: {len(self.df_merged.columns)}")
            
            # Remover duplicados se solicitado
            if remover_duplicados:
                antes = len(self.df_merged)
                # Não considerar a coluna origem_arquivo para detectar duplicados
                cols_para_verificar = [col for col in self.df_merged.columns if col != 'origem_arquivo']
                
                if cols_para_verificar:  # Se houver colunas para verificar
                    self.df_merged = self.df_merged.drop_duplicates(subset=cols_para_verificar, keep='first')
                    depois = len(self.df_merged)
                    if antes > depois:
                        print(f"   🗑️ {antes - depois:,} linhas duplicadas removidas")
            
            # Estatísticas finais
            print(f"\n📊 RESULTADO FINAL:")
            print(f"   Linhas do arquivo 1: {len(self.df1):,}")
            print(f"   Linhas do arquivo 2: {len(self.df2):,}")
            print(f"   Linhas no arquivo unificado: {len(self.df_merged):,}")
            print(f"   Colunas no arquivo unificado: {len(self.df_merged.columns)}")
            
            # Mostrar distribuição por arquivo de origem
            print(f"\n📍 DISTRIBUIÇÃO POR ARQUIVO DE ORIGEM:")
            contagem_origem = self.df_merged['origem_arquivo'].value_counts()
            for origem, count in contagem_origem.items():
                print(f"   • {origem}: {count:,} linhas ({count/len(self.df_merged)*100:.1f}%)")
            
            return True
            
        except Exception as e:
            print(f"❌ Erro ao unir arquivos: {e}")
            import traceback
            traceback.print_exc()
            return False
    
    def salvar_resultado(self, nome_arquivo="ativos e inativos_full.csv", separador='|'):
        """
        Salva o arquivo unificado
        
        Args:
            nome_arquivo: Nome do arquivo de saída
            separador: Separador a usar no arquivo de saída
        """
        if self.df_merged is None:
            print("❌ Nenhum arquivo unificado para salvar!")
            return False
            
        try:
            print(f"\n💾 SALVANDO ARQUIVO UNIFICADO...")
            print("=" * 60)
            
            # Salvar com encoding UTF-8
            self.df_merged.to_csv(nome_arquivo, sep=separador, index=False, encoding='utf-8-sig')
            
            # Verificar arquivo salvo
            if Path(nome_arquivo).exists():
                tamanho = Path(nome_arquivo).stat().st_size / 1024 / 1024
                
                print(f"✅ Arquivo salvo com sucesso!")
                print(f"   📁 Nome: {nome_arquivo}")
                print(f"   📊 Tamanho: {tamanho:.2f} MB")
                print(f"   📝 Linhas: {len(self.df_merged):,}")
                print(f"   📋 Colunas: {len(self.df_merged.columns)}")
                print(f"   🔤 Separador: '{separador}'")
                print(f"   🔠 Encoding: UTF-8 com BOM")
            
            return True
            
        except Exception as e:
            print(f"❌ Erro ao salvar arquivo: {e}")
            import traceback
            traceback.print_exc()
            return False
    
    def executar_processo_completo(self):
        """Executa o processo completo de verificação e união"""
        print("🚀 INICIANDO PROCESSO DE VERIFICAÇÃO E UNIÃO DE CSVs")
        print("=" * 60)
        
        try:
            # 1. Carregar arquivos
            if not self.carregar_arquivos():
                return False
            
            # 2. Analisar estrutura
            if not self.analisar_estrutura():
                print("\n❌ Processo interrompido: arquivos incompatíveis")
                return False
            
            # 3. Verificar tipos de dados
            self.verificar_tipos_dados()
            
            # 4. Mostrar estatísticas
            self.estatisticas_detalhadas()
            
            # 5. Unir arquivos
            sucesso_uniao = self.unir_arquivos()
            
            if sucesso_uniao:
                # 6. Salvar resultado
                self.salvar_resultado()
                print("\n✨ PROCESSO CONCLUÍDO COM SUCESSO!")
                return True
            else:
                print("\n❌ Falha na união dos arquivos")
                return False
                
        except Exception as e:
            print(f"\n❌ Erro no processo: {e}")
            import traceback
            traceback.print_exc()
            return False

# ============================================
# FUNÇÕES AUXILIARES
# ============================================

def verificar_e_unir_csvs(arquivo1, arquivo2, separador='|', nome_saida="ativos_e_inativos_full.csv"):
    """
    Função simplificada para verificar e unir dois CSVs
    
    Args:
        arquivo1: Caminho do primeiro arquivo
        arquivo2: Caminho do segundo arquivo
        separador: Separador usado nos CSVs
        nome_saida: Nome do arquivo de saída
    
    Returns:
        DataFrame unificado ou None se incompatível
    """
    merger = CSVMerger(arquivo1, arquivo2, separador)
    if merger.executar_processo_completo():
        return merger.df_merged
    return None

def analisar_multiplos_csvs(*arquivos, separador='|'):
    """
    Analisa múltiplos arquivos CSV para verificar compatibilidade
    
    Args:
        *arquivos: Lista de caminhos de arquivos
        separador: Separador usado nos CSVs
    """
    print("📊 ANÁLISE DE MÚLTIPLOS ARQUIVOS CSV")
    print("=" * 60)
    
    dataframes = []
    estruturas = []
    
    for i, arquivo in enumerate(arquivos, 1):
        try:
            df = pd.read_csv(arquivo, sep=separador, low_memory=False)
            dataframes.append(df)
            estruturas.append(set(df.columns))
            print(f"✅ Arquivo {i}: {Path(arquivo).name}")
            print(f"   Linhas: {len(df):,}, Colunas: {len(df.columns)}")
        except Exception as e:
            print(f"❌ Erro no arquivo {i}: {e}")
            continue
    
    if not estruturas:
        print("❌ Nenhum arquivo pôde ser carregado")
        return None
    
    # Verificar compatibilidade geral
    colunas_comuns = set.intersection(*estruturas) if estruturas else set()
    print(f"\n📋 Colunas em comum em TODOS os arquivos: {len(colunas_comuns)}")
    
    if len(colunas_comuns) > 0:
        print("✅ Arquivos podem ser unidos!")
        return dataframes
    else:
        print("❌ Arquivos incompatíveis para união")
        return None

# ============================================
# EXEMPLO DE USO
# ============================================

# Exemplo básico - SUBSTITUA COM SEUS ARQUIVOS
print("📝 EXEMPLO DE USO:")
print("=" * 60)
print("""
# Para usar com seus arquivos, execute:

merger = CSVMerger('arquivo1.csv', 'arquivo2.csv', separador='|')
merger.executar_processo_completo()

# Ou use a função simplificada:
df_unificado = verificar_e_unir_csvs(
    'caminho/para/arquivo1.csv',
    'caminho/para/arquivo2.csv',
    separador='|',
    nome_saida='ativos e inativos_full.csv'
)

# Para acessar o DataFrame resultado após a execução:
# df_resultado = merger.df_merged
""")
merger = CSVMerger(r"D:\____________________FAPERGS\DADOS\1_dados_limpos\completo_ativos_com_sobreviveu.csv", r"D:\____________________FAPERGS\DADOS\1_dados_limpos\completo_inativos_nao_sobreviveu_pipe.csv", separador='|')
merger.executar_processo_completo()

# ============================================
# CRIAR ARQUIVOS DE EXEMPLO PARA TESTE
# ============================================

def criar_arquivos_exemplo():
    """Cria arquivos de exemplo para teste"""
    
    # Criar arquivo 1 - Ativos
    df_ativos = pd.DataFrame({
        'cnpj': ['06.303.914/0001-08', '11.222.333/0001-44', '99.888.777/0001-66'],
        'username': ['gruporezor', 'empresa2', 'empresa3'],
        'name': ['Grupo Rezor', 'Empresa 2', 'Empresa 3'],
        'followers_count': [955, 1200, 450],
        'status': ['ativo', 'ativo', 'ativo'],
        'timestamp': ['2025-06-04T18:04:48+0000', '2025-06-05T10:00:00+0000', '2025-06-06T14:30:00+0000']
    })
    
    # Criar arquivo 2 - Inativos
    df_inativos = pd.DataFrame({
        'cnpj': ['55.444.333/0001-22', '77.666.555/0001-11'],
        'username': ['empresa4', 'empresa5'],
        'name': ['Empresa 4', 'Empresa 5'],
        'followers_count': [200, 150],
        'status': ['inativo', 'inativo'],
        'timestamp': ['2025-01-15T09:00:00+0000', '2025-02-20T11:30:00+0000']
    })
    
    # Salvar arquivos de exemplo
    df_ativos.to_csv('exemplo_ativos.csv', sep='|', index=False)
    df_inativos.to_csv('exemplo_inativos.csv', sep='|', index=False)
    
    print("📁 Arquivos de exemplo criados:")
    print("   • exemplo_ativos.csv")
    print("   • exemplo_inativos.csv")
    print("\n🚀 Execute o código abaixo para testá-los:")
    print("merger = CSVMerger('exemplo_ativos.csv', 'exemplo_inativos.csv', '|')")
    print("merger.executar_processo_completo()")
    
    return df_ativos, df_inativos

# Descomentar para criar arquivos de exemplo
# df_ativos, df_inativos = criar_arquivos_exemplo()

print("\n✨ Código pronto para uso! Substitua os nomes dos arquivos e execute.")

📝 EXEMPLO DE USO:

# Para usar com seus arquivos, execute:

merger = CSVMerger('arquivo1.csv', 'arquivo2.csv', separador='|')
merger.executar_processo_completo()

# Ou use a função simplificada:
df_unificado = verificar_e_unir_csvs(
    'caminho/para/arquivo1.csv',
    'caminho/para/arquivo2.csv',
    separador='|',
    nome_saida='ativos e inativos_full.csv'
)

# Para acessar o DataFrame resultado após a execução:
# df_resultado = merger.df_merged

🚀 INICIANDO PROCESSO DE VERIFICAÇÃO E UNIÃO DE CSVs
📂 CARREGANDO ARQUIVOS...
✅ Arquivo 1 carregado com encoding utf-8
   📁 completo_ativos_com_sobreviveu.csv
   📊 Dimensões: 1,292,775 linhas x 15 colunas
✅ Arquivo 2 carregado com encoding utf-8
   📁 completo_inativos_nao_sobreviveu_pipe.csv
   📊 Dimensões: 239,713 linhas x 2 colunas

📋 ANÁLISE DA ESTRUTURA

🔤 COLUNAS:
Arquivo 1: 15 colunas
Arquivo 2: 2 colunas

✅ Colunas em comum: 1
   • sobreviveu

⚠️ Colunas exclusivas do Arquivo 1: 14
   • arquivo_origem
   • biography
   • business_acco